## Install and import

In [1]:
!pip install torch==1.4.0
!pip install transformers==2.9.0
!pip install pytorch_lightning==0.7.5

     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 645kB 15.4MB/s 
     |████████████████████████████████| 5.6MB 16.9MB/s 
     |████████████████████████████████| 1.2MB 53.6MB/s 
     |████████████████████████████████| 901kB 52.5MB/s 
     |████████████████████████████████| 235kB 13.9MB/s 
     |████████████████████████████████| 829kB 26.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=dbe646c71acc646d21f5bcf07db2350fa1effeb8f2e568cb14be23363bc23fcc
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397

In [11]:
import os
import google.colab as colab
import pandas as pd

In [3]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

ROOT_DIR =  mount_google_drive() + "/goemotion/"
CHECKPOINT_ROOT = ROOT_DIR+ "goemotion-ckpt/original/checkpoint-10000"

Mounted at /content/gdrive


# Run inference with any question as input

In [8]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

sentence = "Which course should I take to get started in data science?"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"

def t5_paraphrase(sentence):
  text =  "paraphrase: " + sentence + " </s>"
  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=10
  )

  # print ("\nOriginal Question ::")
  # print (sentence)
  # print ("\n")
  # print ("Paraphrased Questions :: ")
  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)
  return final_outputs

  # for i, final_output in enumerate(final_outputs):
  #     print("{}: {}".format(i, final_output))

device  cuda


In [9]:
t5_paraphrase(sentence)

['What courses should I take to get started in data science?',
 "I'm a data scientist. How can I start learning to be involved in data science?",
 'What are some of the best ways to get started in data science?',
 'What are the best online courses for data science?',
 'What are some good courses to get started in data science?',
 'What are the best data science courses?',
 'How should we start learning data science?',
 'What course should I take if I want to master data science?',
 'What are the best software applications for data science?',
 'What are good online courses for Data Science?']

# Load test set and apply

In [12]:
df = pd.read_csv(ROOT_DIR + "data/original/test.tsv", sep='\t', header=None, names=["sentence", "label", "encoding"])
sentences = df.sentence.tolist()

In [13]:
len(sentences)

5427

In [ ]:
test_sent_to_t5 = {}
for test_sent in sentences:
  test_sent_to_t5[test_sent] = t5_paraphrase(test_sent)